## Python Notebook for a general lens searchquery of the SLED database 

In [ ]:
import os
import json
import requests
from requests.auth import HTTPBasicAuth

from astropy import units as u
from astropy.coordinates import SkyCoord

### Let's define the specific API endpoint and load credentials
You should NOT save your passwords in Python documents. Instead it is safest to either set them as environment variables, for example in your shell profile file (e.g., in Unix/Linux/Mac include export SLED_PASSWORD=??? and export SLED_USERNAME=??? in your .bash_profile), or in an environment file.

In [ ]:
urlquery = "https://sled.amnh.org/api/query-lenses-full/"
username, password = os.getenv('SLED_USERNAME'), os.getenv('SLED_PASSWORD')

if (not username) or (not password):
    print('You have not set your SLED username and password environment variables. Please read the comments above.')

### Define your coordinate positions and search radius for a cone-search
Coordinates should be in decimal degrees, and in the J2000 system. To convert from ra/dec in hours-minutes/degrees-minutes, you can use the following astropy coords functionality.

In [ ]:
position = '17 21 45.1 +88 42 22.0' #or '17:21:45.1 +88:42:22' or '17h21m45.1s +88d42m22s'
c = SkyCoord(position, unit=(u.hourangle, u.deg))

ra, dec = c.ra.degree, c.dec.degree
print(ra,dec)

In [ ]:
radius = 10. #in arcseconds

### Now we query the database by sending the relevant information in a dictionary
Use the requests package to send a POST request to the relevant API endpoint, and parse the response as a json, which will contain a single key 'lenses', the contents of which will be a list of dictionaries corresponding to database matches within the radius of the search position

In [ ]:
#lensdata = {'lens-ra_centre':ra, 'lens-dec_centre':dec, 'lens-radius':10.}
lensdata = {'lens-ra_centre':ra,
            'lens-dec_centre':dec,
            'lens-radius':10.,
            'download_choices-redshift': 1,
            'download_choices-imaging': 0,
            'download_choices-spectrum': 0,
            'download_choices-catalogue': 0,
            'download_choices-genericimage': 0,
            'download_choices-papers': 0,
            'download_choices-model': 0
           }

r = requests.post(urlquery, data=lensdata, auth=HTTPBasicAuth(username, password))

dbquery = json.loads(r.text)

In [ ]:
lenses = dbquery['lenses']

In [ ]:
print('Your search returned', len(lenses), 'SLED match(es) with the following names:')

In [ ]:
print(lenses[0])
for lens in lenses:
    print(lens['name'])

### Here are all the keys returned for the main lens metadata table. Please see other example documents on how to return values from the fields of other tables, such as papers

In [ ]:
if len(lenses):
    for key in lenses[0].keys():
        print(key,'=',lenses[0][key])